<br><br>
<center><u><H1>Image Web Scraping with Selenium</H1></u></center>

In [10]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests

Download Chrom Driver https://chromedriver.chromium.org/home
Check Path images_path = f'./8img/' exist

In [8]:
DRIVER_PATH = 'C:/Users/Chatr/Downloads/chromedriver_win32/chromedriver.exe'

def image_urls(string, max_links, driver):
    
    def scroll_to_end(driver):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)#sleep_between_interactions    
    
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    driver.get(search_url.format(q=string))

    image_urls = set()
    image_count = 0
    results_start = 0
    
    while image_count < max_links:
        scroll_to_end(driver)

        thumbnail_results = driver.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            
            try:
                img.click()
                time.sleep(1)
            except Exception:
                continue
  
            actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if image_count >= max_links:
                print(f"Found: {image_count} image links")
                break
        else:
            print("Found:", image_count, "looking for more image links ...")
            time.sleep(30)
            return
            
            load_more_button = driver.find_element_by_css_selector(".mye4qd")
            
            if load_more_button:
                driver.execute_script("document.querySelector('.mye4qd').click();")

        results_start = len(thumbnail_results)

    return image_urls

def save_images(folder_path,file_name,url):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

In [9]:
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
search_names = ["bus","cars"]
images_path = f'./8img/'

for name in search_names:
    path = os.path.join(images_path,name)
    if not os.path.isdir(path):
        os.mkdir(path)

    driver.get('https://google.com')
    search_box = driver.find_element_by_css_selector('input.gLFyf')
    search_box.send_keys(name)
    links = image_urls(name,5,driver)

    for i, link in enumerate(links):
        file_name = f"{i:06}.jpg"
        save_images(path,file_name,link)
driver.quit()

C:\Users\Chatr\AppData\Local\Temp\ipykernel_21296\2278161733.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)
C:\Users\Chatr\AppData\Local\Temp\ipykernel_21296\2278161733.py:11: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search_box = driver.find_element_by_css_selector('input.gLFyf')
C:\Users\Chatr\AppData\Local\Temp\ipykernel_21296\3582066582.py:20: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  thumbnail_results = driver.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\Chatr\AppData\Local\Temp\ipykernel_21296\3582066582.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = driver.find_elements_by_css_selector('img.n3VNCb')


Found: 5 image links


C:\Users\Chatr\anaconda3\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


SAVED - https://cms.eichertrucksandbuses.com/uploads/truck/sub-category/a933e5958e4a354cfb8d22665bd244fd.png - AT: ./8img/bus\000000.jpg
SAVED - https://www.eichertrucksandbuses.com/front/images/buses_img01.png - AT: ./8img/bus\000001.jpg
SAVED - https://i.pinimg.com/originals/0f/0b/2c/0f0b2cfb4d43228e8d4ef10476ffdd9a.png - AT: ./8img/bus\000002.jpg
SAVED - https://assetsw.bus.com/content/uploads/2020/02/14170850/Charter-Bus-Rentals-Photo.jpg - AT: ./8img/bus\000003.jpg
SAVED - https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/LT_471_%28LTZ_1471%29_Arriva_London_New_Routemaster_%2819522859218%29.jpg/1200px-LT_471_%28LTZ_1471%29_Arriva_London_New_Routemaster_%2819522859218%29.jpg - AT: ./8img/bus\000004.jpg
Found: 100 search results. Extracting links from 0:100
Found: 5 image links
SAVED - https://prod-ripcut-delivery.disney-plus.net/v1/variant/disney/E8AE6930B0A452C2FB77C15CFC0F5C0AE3C0FED690FED059698C650B11EA6BFF/scale?width=1200&aspectRatio=1.78&format=jpeg - AT: ./8img/cars\

## Reference:
- https://selenium-python.readthedocs.io/installation.html